Setup and Data loading

In [2]:
# Data processing
import pandas as pd
# Import files
from google.colab import files
# Upload file
uploaded = files.upload()

Saving 1pharma-data.csv to 1pharma-data (1).csv


Create a data frame and view basic info

In [3]:
df = pd.read_csv('1pharma-data.csv')

# View basic info
print("Shape of dataset:", df.shape)
df.head()


Shape of dataset: (254082, 18)


,Distributor,Customer Name,City,Country,Latitude,Longitude,Channel,Sub-channel,Product_ID,Product Class,Quantity,Price,Sales,Month,Year,Name of Sales Rep,Manager,Sales Team
0,Gottlieb-Cruickshank,"Zieme, Doyle and Kunze",Lublin,Poland,51.2333,22.5667,Hospital,Private,Topipizole,Mood Stabilizers,4.0,368,1472.0,January,2018,Mary Gerrard,Britanny Bold,Delta
1,Gottlieb-Cruickshank,Feest PLC,Świecie,Poland,53.4167,18.4333,Pharmacy,Retail,Choriotrisin,Antibiotics,7.0,591,4137.0,January,2018,Jessica Smith,Britanny Bold,Delta
2,Gottlieb-Cruickshank,Medhurst-Beer Pharmaceutical Limited,Rybnik,Poland,50.0833,18.5000,Pharmacy,Institution,Acantaine,Antibiotics,30.0,66,1980.0,January,2018,Steve Pepple,Tracy Banks,Bravo
3,Gottlieb-Cruickshank,Barton Ltd Pharma Plc,Czeladź,Poland,50.3333,19.0833,Hospital,Private,Lioletine Refliruvax,Analgesics,6.0,435,2610.0,January,2018,Mary Gerrard,Britanny Bold,Delta
4,Gottlieb-Cruickshank,Keeling LLC Pharmacy,Olsztyn,Poland,53.7800,20.4942,Pharmacy,Retail,Oxymotroban Fexoformin,Analgesics,20.0,458,9160.0,January,2018,Anne Wu,Britanny Bold,Delta


Data Exploration

In [4]:
# Check for missing values
print(df.isnull().sum())

# Data types
print(df.dtypes)

# Descriptive statistics
print(df.describe(include='all'))

# Unique values in each column
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")


Distributor          0
Customer Name        0
City                 0
Country              0
Latitude             0
Longitude            0
Channel              0
Sub-channel          0
Product_ID           0
Product Class        0
Quantity             0
Price                0
Sales                0
Month                0
Year                 0
Name of Sales Rep    0
Manager              0
Sales Team           0
dtype: int64
Distributor           object
Customer Name         object
City                  object
Country               object
Latitude             float64
Longitude            float64
Channel               object
Sub-channel           object
Product_ID            object
Product Class         object
Quantity             float64
Price                  int64
Sales                float64
Month                 object
Year                   int64
Name of Sales Rep     object
Manager               object
Sales Team            object
dtype: object
         Distributor                 

Data Cleaning

In [5]:
# Drop duplicates
df.drop_duplicates(inplace=True)

# Fill or drop missing values
df.ffill(inplace=True)  # or choose other imputation methods

# Convert categorical columns if needed
categorical_cols = df.select_dtypes(include='object').columns
df[categorical_cols] = df[categorical_cols].astype('category')


Feature Engineering

In [6]:
# One-hot encoding
df_encoded = pd.get_dummies(df, drop_first=True)

# Example: Create interaction term (adjust based on context)
# df_encoded['new_feature'] = df_encoded['feature1'] * df_encoded['feature2']


XGBoost and LightGBM Modelling

In [24]:
#import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor  # Import LightGBM
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define features and target
X = df_encoded.drop('Quantity', axis=1)
y = df_encoded['Quantity']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Function to replace special characters in column names (for LightGBM) ---
def clean_column_names(df):
    df.columns = [re.sub(r"[^a-zA-Z0-9_]", "_", col) for col in df.columns]
    return df

# --- Clean column names in X_train and X_test (for LightGBM) ---
X_train_lgb = clean_column_names(X_train.copy())
X_test_lgb = clean_column_names(X_test.copy())

# --- Function to calculate metrics ---
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    non_zero_mask = y_true != 0
    mape = np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
    return mse, rmse, mae, r2, mape

# --- WRMSSE Calculation ---
def calculate_wrmsse(df_test, group_col, df):
    def compute_scale(series):
        diff = series.diff().dropna()
        return np.mean(diff ** 2)

    scales = (
        df.groupby(group_col, observed=False)['Quantity']
        .apply(compute_scale)
        .replace(0, np.nan)
    )

    def rmsse(actual, pred, scale):
        return np.sqrt(np.mean((actual - pred) ** 2) / scale)

    errors = []
    weights = []

    for group in df_test[group_col].unique():
        actual = df_test[df_test[group_col] == group]['Actual']
        pred = df_test[df_test[group_col] == group]['Predicted']
        scale = scales.get(group, np.nan)

        if pd.isna(scale) or scale == 0:
            continue

        weight = actual.sum()
        score = rmsse(actual, pred, scale)

        errors.append(score * weight)
        weights.append(weight)

    wrmsse = np.sum(errors) / np.sum(weights) if weights else np.nan
    return wrmsse

# --- Train and evaluate XGBoost ---
model_xgb = XGBRegressor()
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)
mse_xgb, rmse_xgb, mae_xgb, r2_xgb, mape_xgb = calculate_metrics(y_test, y_pred_xgb)

# Create df_test for XGBoost
df_test_xgb = X_test.copy()
df_test_xgb['Actual'] = y_test
df_test_xgb['Predicted'] = y_pred_xgb
df_test_xgb['Product_ID'] = df.loc[X_test.index, 'Product_ID']
wrmsse_xgb = calculate_wrmsse(df_test_xgb, 'Product_ID', df)

# --- Train and evaluate LightGBM ---
model_lgb = LGBMRegressor()
model_lgb.fit(X_train_lgb, y_train)  # Use cleaned data for LightGBM
y_pred_lgb = model_lgb.predict(X_test_lgb)  # Use cleaned data for LightGBM
mse_lgb, rmse_lgb, mae_lgb, r2_lgb, mape_lgb = calculate_metrics(y_test, y_pred_lgb)

# Create df_test for LightGBM
df_test_lgb = X_test_lgb.copy()  # Use cleaned data for LightGBM
df_test_lgb['Actual'] = y_test
df_test_lgb['Predicted'] = y_pred_lgb
df_test_lgb['Product_ID'] = df.loc[X_test.index, 'Product_ID']
wrmsse_lgb = calculate_wrmsse(df_test_lgb, 'Product_ID', df)

# --- Print metrics for both models ---
print("\nXGBoost Metrics:")
print(f"MAE  : {mae:.2f}")
print(f"MSE  : {mse:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R²   : {r2:.2f}")
print(f"MAPE : {mape:.2f}%")
print(f"WRMSSE: {wrmsse:.4f}")

# ... (Print LightGBM metrics) ...
print("\nLightGBM Metrics:")
print(f"MAE  : {mae_lgb:.2f}")
print(f"MSE  : {mse_lgb:.2f}")
print(f"RMSE : {rmse_lgb:.2f}")
print(f"R²   : {r2_lgb:.2f}")
print(f"MAPE : {mape_lgb:.2f}%")
print(f"WRMSSE: {wrmsse_lgb:.4f}")

# --- Visualization ---

# Store metric results in a dictionary
results = {
    'Metric': ['MAE', 'MSE', 'RMSE', 'R²', 'MAPE', 'WRMSSE'],
    'XGBoost': [mae_xgb, mse_xgb, rmse_xgb, r2_xgb, mape_xgb, wrmsse_xgb],
    'LightGBM': [mae_lgb, mse_lgb, rmse_lgb, r2_lgb, mape_lgb, wrmsse_lgb]
}

# --- Determine the best model ---

# Store metric results in a dictionary
results = {
    'Metric': ['MAE', 'MSE', 'RMSE', 'R²', 'MAPE', 'WRMSSE'],
    'XGBoost': [mae_xgb, mse_xgb, rmse_xgb, r2_xgb, mape_xgb, wrmsse_xgb],
    'LightGBM': [mae_lgb, mse_lgb, rmse_lgb, r2_lgb, mape_lgb, wrmsse_lgb]
}

# Create a pandas DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Set the 'Metric' column as the index
results_df.set_index('Metric', inplace=True)

# Find the best model for each metric (excluding R²)
best_models = {}
for metric in results_df.index:
    if metric != 'R²':  # Exclude R² since lower is better
        best_models[metric] = results_df.loc[metric].idxmin()  # Find minimum value for other metrics
    else:
        best_models[metric] = results_df.loc[metric].idxmax()  # Find maximum value for R²

# Print the best models for each metric
for metric, model in best_models.items():
    print(f"Best model for {metric}: {model}")

# Overall best model (based on the most frequent best model)
overall_best_model = pd.Series(best_models).value_counts().idxmax()
print(f"\nOverall best model: {overall_best_model}")




[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.290436 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4588
[LightGBM] [Info] Number of data points in the train set: 203262, number of used features: 1809
[LightGBM] [Info] Start training from score 112.943526

XGBoost Metrics:
MAE  : 22.15
MSE  : 243930.23
RMSE : 493.89
R²   : 0.54
MAPE : 31.35%
WRMSSE: 0.3053

LightGBM Metrics:
MAE  : 25.87
MSE  : 215058.92
RMSE : 463.74
R²   : 0.59
MAPE : 66.01%
WRMSSE: 0.3317
Best model for MAE: XGBoost
Best model for MSE: LightGBM
Best model for RMSE: LightGBM
Best model for R²: LightGBM
Best model for MAPE: XGBoost
Best model for WRMSSE: XGBoost

Overall best model: XGBoost
